## 1. Install and Import Required Libraries

In [36]:
# Install missing package for Jupyter (magic command)
%pip install catboost

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import (
    accuracy_score, roc_auc_score, classification_report,
    confusion_matrix, log_loss, brier_score_loss, roc_curve
)
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

print("All packages loaded successfully!")

Note: you may need to restart the kernel to use updated packages.
All packages loaded successfully!
Note: you may need to restart the kernel to use updated packages.
All packages loaded successfully!


## 2. Load L10 Dataset (Best Performing Window)

In [37]:
# Load the L10 dataset (confirmed as best from LASSO analysis)
df_L10 = pd.read_csv('nba_matchups_with_features_L10.csv')

# Convert date columns
df_L10['date'] = pd.to_datetime(df_L10['date'])

print(f"Dataset loaded: {len(df_L10):,} matchups")
print(f"Seasons: {sorted(df_L10['season'].unique())}")
print(f"\nGames per season:")
print(df_L10['season'].value_counts().sort_index())

Dataset loaded: 5,145 matchups
Seasons: [np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]

Games per season:
season
2022    1290
2023    1288
2024    1285
2025    1282
Name: count, dtype: int64


## 3. Define Features and Create Train/Test Splits

In [38]:
def get_features_L10():
    """
    Return list of base features for L10 window.
    Features are differential (away - home) unless noted.
    """
    features = [
        # Advanced Efficiency Gaps
        'off_rtg_L10_diff',       # Offensive efficiency gap
        'def_rtg_L10_diff',       # Defensive efficiency gap
        'net_rtg_L10_diff',       # Net rating gap
        
        # Shooting Efficiency Gaps
        'efg_pct_L10_diff',       # Effective FG% gap
        '3p_pct_L10_diff',        # 3-point shooting gap
        '3pa_rate_L10_diff',      # 3-point volume gap
        
        # Form/Momentum
        'win_pct_L10_diff',       # Recent win percentage gap
        
        # Ball Control
        'to_pct_L10_diff',        # Turnover rate gap
        'ft_rate_L10_diff',       # Free throw rate gap
        
        # Rebounding & Playmaking
        'oreb_pct_L10_diff',      # Offensive rebounding gap
        'ast_ratio_L10_diff',     # Assist ratio gap
        
        # Defensive Stats
        'stl_pct_L10_diff',       # Steal percentage gap
        'blk_pct_L10_diff',       # Block percentage gap
        
        # Consistency & Momentum (non-window features)
        'pts_std_L10_diff',       # Scoring consistency gap
        'win_streak_diff',        # Win streak differential
        
        # Rest/Fatigue
        'rest_advantage',         # Rest days advantage
        'is_b2b_home',            # Home team back-to-back
        'is_b2b_away'             # Away team back-to-back
    ]
    
    return features

features_L10 = get_features_L10()

print(f"Base features (L10): {len(features_L10)}")
print("\nFeatures:")
for i, f in enumerate(features_L10, 1):
    print(f"  {i:2d}. {f}")

Base features (L10): 18

Features:
   1. off_rtg_L10_diff
   2. def_rtg_L10_diff
   3. net_rtg_L10_diff
   4. efg_pct_L10_diff
   5. 3p_pct_L10_diff
   6. 3pa_rate_L10_diff
   7. win_pct_L10_diff
   8. to_pct_L10_diff
   9. ft_rate_L10_diff
  10. oreb_pct_L10_diff
  11. ast_ratio_L10_diff
  12. stl_pct_L10_diff
  13. blk_pct_L10_diff
  14. pts_std_L10_diff
  15. win_streak_diff
  16. rest_advantage
  17. is_b2b_home
  18. is_b2b_away


In [39]:
def create_train_test_split(df, features, target='win_away'):
    """
    Split data into train (≤2024) and test (2025) sets.
    Returns X_train, X_test, y_train, y_test, dates
    """
    # Verify all features exist
    missing = [f for f in features if f not in df.columns]
    if missing:
        print(f"Warning: Missing features: {missing}")
        features = [f for f in features if f in df.columns]
    
    # Drop any rows with missing values in features or target
    df_clean = df.dropna(subset=features + [target])
    
    # Split by season
    train_mask = df_clean['season'] <= 2024
    test_mask = df_clean['season'] == 2025
    
    X_train = df_clean.loc[train_mask, features].copy()
    X_test = df_clean.loc[test_mask, features].copy()
    y_train = df_clean.loc[train_mask, target].copy()
    y_test = df_clean.loc[test_mask, target].copy()
    
    # Store dates for analysis
    dates_train = df_clean.loc[train_mask, 'date'].copy()
    dates_test = df_clean.loc[test_mask, 'date'].copy()
    
    print(f"Train: {len(X_train):,} games (seasons ≤ 2024)")
    print(f"Test:  {len(X_test):,} games (season 2025)")
    print(f"Train away win rate: {y_train.mean():.1%}")
    print(f"Test away win rate:  {y_test.mean():.1%}")
    
    return X_train, X_test, y_train, y_test, dates_train, dates_test, features

# Create split (2022-2024 for model training/calib, 2025 for final test)
print("=" * 60)
print("L10 Dataset Split:")
print("=" * 60)
X_train, X_test, y_train, y_test, dates_train, dates_test, features = create_train_test_split(
    df_L10, features_L10
)

# Further split training data into model training (75%) and calibration (25%)
print("\n" + "=" * 60)
print("Three-Way Split (Training + Calibration + Test):")
print("=" * 60)

split_idx = int(len(X_train) * 0.75)
X_train_model = X_train.iloc[:split_idx]
y_train_model = y_train.iloc[:split_idx]
dates_train_model = dates_train.iloc[:split_idx]

X_calib = X_train.iloc[split_idx:]
y_calib = y_train.iloc[split_idx:]
dates_calib = dates_train.iloc[split_idx:]

print(f"\nTraining (model fit + CV):   {len(X_train_model):,} games (75%)")
print(f"Calibration (isotonic fit): {len(X_calib):,} games (25%)")
print(f"Test (final eval):          {len(X_test):,} games (2025)")
print(f"\nTraining away win rate: {y_train_model.mean():.1%}")
print(f"Calibration away win rate: {y_calib.mean():.1%}")
print(f"Test away win rate: {y_test.mean():.1%}")

L10 Dataset Split:
Train: 3,863 games (seasons ≤ 2024)
Test:  1,282 games (season 2025)
Train away win rate: 44.1%
Test away win rate:  45.2%

Three-Way Split (Training + Calibration + Test):

Training (model fit + CV):   2,897 games (75%)
Calibration (isotonic fit): 966 games (25%)
Test (final eval):          1,282 games (2025)

Training away win rate: 42.3%
Calibration away win rate: 49.4%
Test away win rate: 45.2%
Train: 3,863 games (seasons ≤ 2024)
Test:  1,282 games (season 2025)
Train away win rate: 44.1%
Test away win rate:  45.2%

Three-Way Split (Training + Calibration + Test):

Training (model fit + CV):   2,897 games (75%)
Calibration (isotonic fit): 966 games (25%)
Test (final eval):          1,282 games (2025)

Training away win rate: 42.3%
Calibration away win rate: 49.4%
Test away win rate: 45.2%


## 4. Train CatBoost Models with Different Hyperparameters

Test multiple hyperparameter configurations and select the best one based on CV performance. Use randomsearch for speed. 

In [40]:
# Note: Using only X_train_model (75%) for hyperparameter search
# X_calib (25%) is reserved exclusively for isotonic regression fitting
print(f"\nHyperparameter search will use: {len(X_train_model):,} games (75%)")
print(f"Isotonic calibration will use: {len(X_calib):,} games (25%)")
print(f"Final evaluation will use: {len(X_test):,} games (2025)")


Hyperparameter search will use: 2,897 games (75%)
Isotonic calibration will use: 966 games (25%)
Final evaluation will use: 1,282 games (2025)


In [41]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

print("=" * 80)
print("HYPERPARAMETER TUNING WITH RANDOMIZED SEARCH")
print("=" * 80)


# Define parameter distribution for random search
param_dist = {
    'depth': randint(2, 7),                    # Range 2-6 (depth of trees)
    'learning_rate': [0.01, 0.05, 0.1],       # Learning rate options
    'iterations': [300, 500, 750, 1000]       # Number of boosting rounds
}

# Create base CatBoost model
base_model = CatBoostClassifier(
    random_state=42,
    loss_function='Logloss',
    eval_metric='AUC',  # Early stopping based on AUC, but we optimize Log Loss
    task_type='CPU',
    verbose=0
)

# Randomized Search with 10 iterations (tests 10 random combinations)
print("\nRunning RandomizedSearchCV with 10 random configurations...")
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,  # Number of random combinations to try
    cv=5,       # 5-fold cross-validation on X_train
    scoring='neg_log_loss',  # Optimize for LOG LOSS (lower is better)
    n_jobs=-1, 
    random_state=42,
    verbose=1
)

# Fit on model training data ONLY (uses 5-fold CV internally for hyperparameter selection)
# Calibration set is held completely separate
random_search.fit(X_train_model, y_train_model)

# Get best model and parameters
best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_model_name = f"Best (depth={best_params['depth']}, lr={best_params['learning_rate']}, iter={best_params['iterations']})"

print("\n" + "=" * 80)
print("HYPERPARAMETER SEARCH RESULTS")
print("=" * 80)
print(f"\nBest Parameters Found: {best_params}")
print(f"Best Cross-Validation Log Loss: {-random_search.best_score_:.4f}")  # Negate because scoring is neg_log_loss

# Evaluate on test set (final evaluation)
print(f"\nEvaluating best model on test set (2025 season)...")
y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]

test_auc = roc_auc_score(y_test, y_test_pred_proba)
test_logloss = log_loss(y_test, y_test_pred_proba)

print(f"\nTest Set Performance (2025 season - FINAL EVALUATION):")
print( "Not Finalized, needs probability calibration before final results")
print(f"  • AUC-ROC:  {test_auc:.4f}")
print(f"  • Log Loss: {test_logloss:.4f}")


HYPERPARAMETER TUNING WITH RANDOMIZED SEARCH

Running RandomizedSearchCV with 10 random configurations...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits

HYPERPARAMETER SEARCH RESULTS

Best Parameters Found: {'depth': 3, 'iterations': 1000, 'learning_rate': 0.01}
Best Cross-Validation Log Loss: 0.6527

Evaluating best model on test set (2025 season)...

Test Set Performance (2025 season - FINAL EVALUATION):
Not Finalized, needs probability calibration before final results
  • AUC-ROC:  0.6871
  • Log Loss: 0.6364

HYPERPARAMETER SEARCH RESULTS

Best Parameters Found: {'depth': 3, 'iterations': 1000, 'learning_rate': 0.01}
Best Cross-Validation Log Loss: 0.6527

Evaluating best model on test set (2025 season)...

Test Set Performance (2025 season - FINAL EVALUATION):
Not Finalized, needs probability calibration before final results
  • AUC-ROC:  0.6871
  • Log Loss: 0.6364


## 5. Probability Calibration: Isotonic Regression on Held-Out Data

CatBoost probabilities can be poorly calibrated. We use Isotonic Regression fit on the held-out calibration set (X_calib, 25% of training data) to fix this.

**Key Principle:** Calibrator is fit on X_calib (never seen during model training), ensuring no data leakage.


In [42]:
print("=" * 80)
print("PROBABILITY CALIBRATION: ISOTONIC REGRESSION")
print("=" * 80)

print("""
Isotonic regression is fit on HELD-OUT data (X_calib).
This ensures the calibration is independent of model training and test evaluation.
""")

# Step 1: Get uncalibrated probabilities on held-out calibration set
calib_probs_uncalibrated = best_model.predict_proba(X_calib)[:, 1]

# Step 2: Fit isotonic regression on held-out calibration set
calibrator = IsotonicRegression(out_of_bounds='clip')
calibrator.fit(calib_probs_uncalibrated, y_calib)
print(f"  Calibrator fitted successfully on {len(X_calib):,} games")

# Step 3: Get uncalibrated test predictions
test_probs_uncalibrated = best_model.predict_proba(X_test)[:, 1]
test_probs_calibrated = calibrator.predict(test_probs_uncalibrated)

print(f"\nUncalibrated test probs - Min: {test_probs_uncalibrated.min():.4f}, Max: {test_probs_uncalibrated.max():.4f}")
print(f"Uncalibrated test probs - Mean: {test_probs_uncalibrated.mean():.4f}, Std: {test_probs_uncalibrated.std():.4f}")
print(f"\nCalibrated test probs - Min: {test_probs_calibrated.min():.4f}, Max: {test_probs_calibrated.max():.4f}")
print(f"Calibrated test probs - Mean: {test_probs_calibrated.mean():.4f}, Std: {test_probs_calibrated.std():.4f}")

print("done")

PROBABILITY CALIBRATION: ISOTONIC REGRESSION

Isotonic regression is fit on HELD-OUT data (X_calib).
This ensures the calibration is independent of model training and test evaluation.

  Calibrator fitted successfully on 966 games

Uncalibrated test probs - Min: 0.1204, Max: 0.7871
Uncalibrated test probs - Mean: 0.4254, Std: 0.1462

Calibrated test probs - Min: 0.0000, Max: 1.0000
Calibrated test probs - Mean: 0.4779, Std: 0.1730
done


## 6. Evaluate Calibrated vs Uncalibrated Performance


In [43]:
print("=" * 80)
print("CALIBRATION EVALUATION: Uncalibrated vs Isotonic-Calibrated")
print("=" * 80)

def evaluate_predictions(y_true, probs_uncalibrated, probs_calibrated):
    """
    Comprehensive evaluation of calibrated vs uncalibrated probabilities.
    """
    results_dict = {
        'Metric': [],
        'Uncalibrated': [],
        'Calibrated': [],
    }
    
    metrics = [
        ('Accuracy', lambda p: accuracy_score(y_true, p > 0.5)),
        ('AUC-ROC', lambda p: roc_auc_score(y_true, p)),
        ('Log Loss', lambda p: log_loss(y_true, p)),
        ('Brier Score', lambda p: brier_score_loss(y_true, p)),
    ]
    #eg. for accuracy, accuracy_score (metric name, metric function)
    for metric_name, metric_fn in metrics:
        #get each score for calibrated/uncalibrated
        uncalibrated_score = metric_fn(probs_uncalibrated)
        calibrated_score = metric_fn(probs_calibrated)

        results_dict['Metric'].append(metric_name)
        results_dict['Uncalibrated'].append(f"{uncalibrated_score:.4f}")
        results_dict['Calibrated'].append(f"{calibrated_score:.4f}")
    
    return pd.DataFrame(results_dict)

print("\nTest Set Performance (2025 season):")
print("Comparing uncalibrated vs isotonic-calibrated predictions\n")

evaluation_df = evaluate_predictions(y_test, test_probs_uncalibrated, test_probs_calibrated)
print("\n" + evaluation_df.to_string(index=False))

print("\n" + "-" * 80)
print("INTERPRETATION:")
print("-" * 80)
print("\nLog Loss & Brier Score (lower is better):")
print("  • Positive improvement = calibration reduced these values (better)")
print("\nAccuracy & AUC (higher is better):")
print("  • Positive improvement = calibration increased these values (better)")
print("\nNote: Calibration primarily improves Log Loss (probability accuracy)")
print("      Accuracy/AUC may not change significantly")

CALIBRATION EVALUATION: Uncalibrated vs Isotonic-Calibrated

Test Set Performance (2025 season):
Comparing uncalibrated vs isotonic-calibrated predictions


     Metric Uncalibrated Calibrated
   Accuracy       0.6381     0.6365
    AUC-ROC       0.6871     0.6804
   Log Loss       0.6364     0.8230
Brier Score       0.2229     0.2254

--------------------------------------------------------------------------------
INTERPRETATION:
--------------------------------------------------------------------------------

Log Loss & Brier Score (lower is better):
  • Positive improvement = calibration reduced these values (better)

Accuracy & AUC (higher is better):
  • Positive improvement = calibration increased these values (better)

Note: Calibration primarily improves Log Loss (probability accuracy)
      Accuracy/AUC may not change significantly
